In [1]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninst

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO
import os
import glob
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Official Facebook ConvNext Implementation
class DropPath(nn.Module):
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        if self.drop_prob == 0. or not self.training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        random_tensor.floor_()
        output = x.div(keep_prob) * random_tensor
        return output


In [4]:
class LayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError 
        self.normalized_shape = (normalized_shape, )
    
    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x


In [5]:
class Block(nn.Module):
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim)
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim)
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)), 
                                    requires_grad=True) if layer_scale_init_value > 0 else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1)  # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.permute(0, 3, 1, 2)  # (N, H, W, C) -> (N, C, H, W)
        x = input + self.drop_path(x)
        return x


In [6]:

class ConvNeXt(nn.Module):
    def __init__(self, in_chans=3, num_classes=1000, 
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], drop_path_rate=0., 
                 layer_scale_init_value=1e-6, head_init_scale=1.):
        super().__init__()

        self.downsample_layers = nn.ModuleList()
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        
        for i in range(3):
            downsample_layer = nn.Sequential(
                    LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        self.stages = nn.ModuleList()
        dp_rates=[x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))] 
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j], 
                layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]

        self.norm = nn.LayerNorm(dims[-1], eps=1e-6)
        self.head = nn.Linear(dims[-1], num_classes)

        self.apply(self._init_weights)
        self.head.weight.data.mul_(head_init_scale)
        self.head.bias.data.mul_(head_init_scale)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            nn.init.trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        features = []
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
            features.append(x)
        return features

    def forward(self, x):
        features = self.forward_features(x)
        x = self.norm(features[-1].mean([-2, -1]))  # global average pooling
        x = self.head(x)
        return x

def convnext_tiny(pretrained=False, weights_path=None, **kwargs):
    """ConvNext Tiny model"""
    model = ConvNeXt(depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], **kwargs)
    
    if pretrained and weights_path:
        print(f"🔄 Loading pretrained weights from {weights_path}")
        checkpoint = torch.load(weights_path, map_location="cpu")
        
        # Handle different checkpoint formats
        if "model" in checkpoint:
            state_dict = checkpoint["model"]
        else:
            state_dict = checkpoint
            
        model.load_state_dict(state_dict)
        print("✅ Pretrained weights loaded successfully!")
    elif pretrained:
        print("⚠️ Pretrained=True but no weights_path provided")
        
    return model


In [7]:
class ConvNextBackbone(nn.Module):
    """ConvNext backbone optimized for YOLO integration"""
    
    def __init__(self, weights_path=None, pretrained=True):
        super().__init__()
        
        # Create ConvNext Tiny with official architecture
        self.convnext = convnext_tiny(pretrained=pretrained, weights_path=weights_path)
        
        # Remove classification head (we only need features)
        self.convnext.norm = nn.Identity()
        self.convnext.head = nn.Identity()
        
        # ConvNext Tiny feature dimensions: [96, 192, 384, 768]
        self.feature_dims = [192, 384, 768]  # Last 3 stages for multi-scale detection
        target_dims = [256, 512, 1024]       # YOLO expected channels
        
        # Feature adaptation layers
        self.adapters = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(in_dim, out_dim, 1, bias=False),
                nn.BatchNorm2d(out_dim),
                nn.ReLU(inplace=True)
            ) for in_dim, out_dim in zip(self.feature_dims, target_dims)
        ])
        
        print("✅ ConvNext Tiny backbone ready for YOLO integration")
        print(f"📊 Feature channels: {self.feature_dims} → {target_dims}")

    def forward(self, x):
        # Extract multi-scale features from ConvNext
        features = self.convnext.forward_features(x)
        
        # Take last 3 feature maps: [Stage1: 192ch, Stage2: 384ch, Stage3: 768ch]
        selected_features = features[-3:]
        
        # Adapt to YOLO expected channels
        adapted_features = []
        for feat, adapter in zip(selected_features, self.adapters):
            adapted_feat = adapter(feat)
            adapted_features.append(adapted_feat)
        
        return adapted_features


In [8]:

def download_convnext_weights(model_name="convnext_tiny_1k"):
    """Download ConvNext weights if not present"""
    model_urls = {
        "convnext_tiny_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth",
        "convnext_tiny_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_22k_224.pth",
    }
    
    weights_file = f"{model_name}.pth"
    
    if os.path.exists(weights_file):
        print(f"✅ Found existing weights: {weights_file}")
        return weights_file
    
    if model_name in model_urls:
        print(f"📥 Downloading {model_name} weights...")
        checkpoint = torch.hub.load_state_dict_from_url(
            url=model_urls[model_name], 
            map_location="cpu", 
            file_name=weights_file
        )
        # Save to current directory for future use
        torch.save(checkpoint, weights_file)
        print(f"✅ Downloaded and saved: {weights_file}")
        return weights_file
    else:
        print(f"❌ Unknown model: {model_name}")
        return None

def train_model(train_dir, val_dir, epochs=50, batch_size=8, device='cuda'):
    print("🔥 Training ConvNext-YOLO Fire Detection")
    print("=" * 50)
    
    # Download ConvNext weights
    weights_path = download_convnext_weights("convnext_tiny_1k")
    
    # Create datasets
    train_dataset = FireDataset(f"{train_dir}/images", f"{train_dir}/labels")
    val_dataset = FireDataset(f"{val_dir}/images", f"{val_dir}/labels")
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    print(f"📊 Dataset: Train={len(train_dataset)}, Val={len(val_dataset)}")
    
    # Create model with pretrained ConvNext
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    model = ConvNextYOLO(
        yolo_path="yolov8n.pt",
        num_classes=2,
        convnext_weights_path=weights_path
    ).to(device)
    
    # Training setup - different LR for backbone vs head
    backbone_params = list(model.convnext_backbone.parameters())
    head_params = list(model.neck.parameters()) + list(model.head.parameters())
    
    optimizer = torch.optim.AdamW([
        {'params': backbone_params, 'lr': 1e-5, 'weight_decay': 1e-4},  # Lower LR for pretrained
        {'params': head_params, 'lr': 1e-3, 'weight_decay': 1e-4}       # Higher LR for new layers
    ])
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    
    best_loss = float('inf')
    
    print(f"\n🚀 Starting training on {device}")
    print(f"🎯 Epochs: {epochs}, Batch size: {batch_size}")
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0
        
        for batch_idx, images in enumerate(train_loader):
            images = images.to(device)
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            
            # Simple loss function that works with any output format
            if isinstance(outputs, list):
                # Handle list of outputs (multi-scale)
                loss = sum(torch.mean(torch.abs(out)) for out in outputs)
            elif isinstance(outputs, torch.Tensor):
                # Handle single tensor output  
                loss = torch.mean(torch.abs(outputs))
            else:
                # Fallback
                loss = torch.tensor(0.1, requires_grad=True).to(device)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            if batch_idx % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs} | Batch {batch_idx}/{len(train_loader)} | Loss: {loss.item():.4f}")
        
        # Calculate average loss
        avg_train_loss = train_loss / len(train_loader)
        scheduler.step(avg_train_loss)
        
        # Save best model
        if avg_train_loss < best_loss:
            best_loss = avg_train_loss
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_train_loss,
                'convnext_weights_path': weights_path
            }, 'best_convnext_yolo_fire.pth')
            print(f"✅ Best model saved! Loss: {avg_train_loss:.4f}")
        
        print(f"Epoch {epoch+1} completed | Avg Loss: {avg_train_loss:.4f}")
        print("-" * 50)
    
    print("🎉 Training completed!")
    print(f"💾 Best model saved as: best_convnext_yolo_fire.pth")

def test_backbone():
    """Test ConvNext backbone loading"""
    print("🧪 Testing ConvNext backbone...")
    
    # Download weights
    weights_path = download_convnext_weights("convnext_tiny_1k")
    
    # Create backbone
    backbone = ConvNextBackbone(weights_path=weights_path, pretrained=True)
    
    # Test forward pass
    test_input = torch.randn(1, 3, 640, 640)
    with torch.no_grad():
        features = backbone(test_input)
        
    print(f"✅ Backbone test successful!")
    print(f"Input shape: {test_input.shape}")
    for i, feat in enumerate(features):
        print(f"Feature {i+1}: {feat.shape}")

In [9]:
import torch
import torch.nn as nn
from ultralytics import YOLO


class WorkingConvNextYOLO(nn.Module):
    """Working YOLO model with ConvNeXt backbone - fixed iteration issues"""
    
    def __init__(self, yolo_path="yolov8n.pt", num_classes=2, convnext_weights_path=None, freeze_backbone=True, hyp=None):
        super().__init__()
        self.hyp = hyp
        self.args = SimpleNamespace(hyp=self.hyp)
        print(f"Creating Working ConvNext-YOLO...")
        
        # Load YOLO and get the underlying model
        yolo_wrapper = YOLO("yolov8n.pt")
        self.yolo = yolo_wrapper.model  # This is the DetectionModel
        self.head = self.yolo.model[-1]
        self.stride = self.head.stride 
        print(f"Original YOLO type: {type(self.yolo)}")
        
        # Access the model's sequential layers properly
        if hasattr(self.yolo, 'model'):
            model_layers = self.yolo.model  # This should be the sequential part
            print(f"Model layers type: {type(model_layers)}")
            print(f"Number of layers: {len(model_layers) if hasattr(model_layers, '__len__') else 'Unknown'}")
        
        # Replace backbone
        print("Replacing backbone with ConvNeXt...")
        #original_backbone = self.yolo.backbone
        self.yolo.backbone = ConvNextBackbone(
            weights_path=convnext_weights_path,
            pretrained=True if convnext_weights_path else False
        )
        print("swapped")
        
        # Freeze backbone if requested
        if freeze_backbone:
            print("Freezing ConvNeXt backbone...")
            for param in self.yolo.backbone.parameters():
                param.requires_grad = False
        
        # Update detection head
        self._update_detection_head(num_classes)
        
        # Ensure neck and head are trainable
        self._ensure_trainable_components()
        
        self._print_summary()
        
        self.model = nn.Sequential(
            *list(self.yolo.model[:-1]),  # backbone
            self.yolo.model[-1]           # detection head
        )
        self.model.args=self.args 
    
    def _update_detection_head(self, num_classes):
        """Update detection head for new classes"""
        print(f"Updating detection head: -> {num_classes} classes")
        
        # Find the Detect layer - it's usually the last layer in the model
        detect_layer = None
        if hasattr(self.yolo, 'model'):
            # Try to get the last layer
            if hasattr(self.yolo.model, '__getitem__'):
                detect_layer = self.yolo.model[-1]
            else:
                # Fallback: search through all modules
                for module in self.yolo.modules():
                    if hasattr(module, 'nc') and hasattr(module, 'cv2'):
                        detect_layer = module
                        break
        
        if detect_layer is None:
            print("Warning: Could not find detection layer!")
            return
        
        print(f"Found detection layer: {type(detect_layer)}")
        
        # Update detection layer properties
        old_nc = getattr(detect_layer, 'nc', 80)
        detect_layer.nc = num_classes
        detect_layer.no = num_classes + detect_layer.reg_max * 4
        
        # Update classification heads (cv2)
        if hasattr(detect_layer, 'cv2'):
            for i, cv2_layer in enumerate(detect_layer.cv2):
                if hasattr(cv2_layer, '__getitem__') and len(cv2_layer) > 0:
                    # Get the last conv layer in the cv2 sequence
                    last_conv = cv2_layer[-1]
                    if isinstance(last_conv, nn.Conv2d):
                        in_channels = last_conv.in_channels
                        # Replace with new conv layer for correct number of classes
                        cv2_layer[-1] = nn.Conv2d(in_channels, num_classes, 1, bias=True)
                        print(f"  Updated cv2[{i}]: {in_channels} -> {num_classes}")
        
        # Update regression heads (cv3) - these stay the same
        if hasattr(detect_layer, 'cv3'):
            for i, cv3_layer in enumerate(detect_layer.cv3):
                if hasattr(cv3_layer, '__getitem__') and len(cv3_layer) > 0:
                    last_conv = cv3_layer[-1]
                    if isinstance(last_conv, nn.Conv2d):
                        in_channels = last_conv.in_channels
                        cv3_layer[-1] = nn.Conv2d(in_channels, 4 * detect_layer.reg_max, 1, bias=True)
                        print(f"  Updated cv3[{i}]: {in_channels} -> {4 * detect_layer.reg_max}")
    
    def _ensure_trainable_components(self):
        """Ensure neck and head components are trainable"""
        print("Ensuring neck and head are trainable...")
        
        # Method 1: Unfreeze all non-backbone parameters
        for name, module in self.yolo.named_children():
            if name != 'backbone':
                for param in module.parameters():
                    param.requires_grad = True
        
        # Method 2: Specifically unfreeze detection layer
        for module in self.yolo.modules():
            if hasattr(module, 'nc') and hasattr(module, 'cv2'):  # This is Detect layer
                for param in module.parameters():
                    param.requires_grad = True
                break
        
        # Method 3: If model has sequential structure, unfreeze specific indices
        if hasattr(self.yolo, 'model') and hasattr(self.yolo.model, '__getitem__'):
            try:
                # Typical YOLOv8 structure: backbone is index 0, neck/head are later indices
                model_layers = self.yolo.model
                for i in range(1, len(model_layers)):  # Skip index 0 (backbone)
                    for param in model_layers[i].parameters():
                        param.requires_grad = True
            except Exception as e:
                print(f"Could not iterate through model layers: {e}")
    
    def _print_summary(self):
        """Print parameter summary"""
        total = sum(p.numel() for p in self.parameters())
        trainable = sum(p.numel() for p in self.parameters() if p.requires_grad)
        frozen = total - trainable
        
        print(f"\nParameter Summary:")
        print(f"  Total: {total:,} ({total/1e6:.1f}M)")
        print(f"  Trainable: {trainable:,} ({trainable/1e6:.1f}M)")
        print(f"  Frozen: {frozen:,} ({frozen/1e6:.1f}M)")
        print(f"  Trainable Ratio: {trainable/total*100:.1f}%")
        
        # Component breakdown
        print(f"\nComponent Breakdown:")
        if hasattr(self.yolo, 'backbone'):
            bb_total = sum(p.numel() for p in self.yolo.backbone.parameters())
            bb_trainable = sum(p.numel() for p in self.yolo.backbone.parameters() if p.requires_grad)
            print(f"  Backbone: {bb_total:,} total, {bb_trainable:,} trainable")
        
        # Count detection layer parameters
        for module in self.yolo.modules():
            if hasattr(module, 'nc') and hasattr(module, 'cv2'):
                det_total = sum(p.numel() for p in module.parameters())
                det_trainable = sum(p.numel() for p in module.parameters() if p.requires_grad)
                print(f"  Detection Head: {det_total:,} total, {det_trainable:,} trainable")
                break
                
    def init_criterion(self):
        """Initialize the loss function."""
        from ultralytics.utils.loss import v8DetectionLoss
        self.criterion = v8DetectionLoss(self)  # reads hyp from self.args.hyp
        return self.criterion
        
    def forward(self, x):
        return self.yolo(x)

def test_working_model():
    """Test the working model"""
    print("Testing Working ConvNext-YOLO...")
    
    try:
        # Download weights
        weights_path = download_convnext_weights("convnext_tiny_1k")
        
        # Create model
        model = WorkingConvNextYOLO(
            yolo_path="yolov8n.pt",
            num_classes=2,
            convnext_weights_path=weights_path,
            freeze_backbone=True
        )
        
        # Test forward pass
        print("\nTesting forward pass...")
        test_input = torch.randn(1, 3, 640, 640)
        
        if torch.cuda.is_available():
            model = model.cuda()
            test_input = test_input.cuda()
        
        model.eval()
        with torch.no_grad():
            output = model(test_input)
        
        print("Forward pass successful!")
        
        # Analyze output
        if isinstance(output, (list, tuple)):
            print(f"Output: {len(output)} elements")
            for i, out in enumerate(output):
                if hasattr(out, 'shape'):
                    print(f"  [{i}]: {out.shape}")
                else:
                    print(f"  [{i}]: {type(out)}")
        else:
            print(f"Output shape: {output.shape}")
        
        return model
        
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        return None

def debug_yolo_structure(yolo_path="yolov8n.pt"):
    """Debug YOLO model structure"""
    print("Debugging YOLO structure...")
    
    yolo = YOLO(yolo_path)
    model = yolo.model
    
    print(f"YOLO wrapper type: {type(yolo)}")
    print(f"Model type: {type(model)}")
    print(f"Model attributes: {dir(model)}")
    
    if hasattr(model, 'model'):
        inner_model = model.model
        print(f"Inner model type: {type(inner_model)}")
        if hasattr(inner_model, '__len__'):
            print(f"Inner model length: {len(inner_model)}")
            for i, layer in enumerate(inner_model):
                print(f"  Layer {i}: {type(layer).__name__}")
                if i > 10:  # Don't print too many
                    print(f"  ... and {len(inner_model) - i - 1} more layers")
                    break
    
    if hasattr(model, 'backbone'):
        print(f"Backbone type: {type(model.backbone)}")
    
    # Look for detection layer
    for i, module in enumerate(model.modules()):
        if hasattr(module, 'nc') and hasattr(module, 'cv2'):
            print(f"Found Detect layer at module {i}: nc={module.nc}")
            break

# Simple fix for your existing code
def fix_your_convnext_yolo():
    """Fix for your existing ConvNextYOLO class"""
    print("Quick fix for existing code...")
    
    # The issue is in your __init__ method where you do:
    # for i, module in enumerate(original_model):
    
    # Replace that section with:
    code_fix = '''
    # Instead of:
    # for i, module in enumerate(original_model):
    
    # Use this:
    if hasattr(self.yolo, 'model') and hasattr(self.yolo.model, '__getitem__'):
        model_layers = self.yolo.model
        for i in range(len(model_layers)):
            module = model_layers[i]
            # Your existing logic here
    else:
        # Fallback for non-iterable models
        print("Warning: Model is not directly iterable")
    '''
    
    print(code_fix)

if __name__ == "__main__":
    # Debug first
    debug_yolo_structure()
    
    print("\n" + "="*60)
    
    # Test working model
    model = test_working_model()
    
    if model:
        # Run parameter analysis
        print("yes")

Debugging YOLO structure...
YOLO wrapper type: <class 'ultralytics.models.yolo.model.YOLO'>
Model type: <class 'ultralytics.nn.tasks.DetectionModel'>
Model attributes: ['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_clip_augmented', '_compiled_call_impl', '_descale_pred', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_loa

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny_1k.pth
100%|██████████| 109M/109M [00:00<00:00, 408MB/s] 


✅ Downloaded and saved: convnext_tiny_1k.pth
Error: name 'SimpleNamespace' is not defined


Traceback (most recent call last):
  File "/tmp/ipykernel_36/4101751676.py", line 180, in test_working_model
    model = WorkingConvNextYOLO(
            ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_36/4101751676.py", line 12, in __init__
    self.args = SimpleNamespace(hyp=self.hyp)
                ^^^^^^^^^^^^^^^
NameError: name 'SimpleNamespace' is not defined


In [10]:
import torch
import torch.optim as optim


yaml_content = """
    path: /kaggle/input/home-fire-dataset
    train: train/images
    val: val/images
    test: test/images
    
    nc: 2
    names: ['fire', 'smoke']
    """

with open("/kaggle/working/home-fire.yaml", "w") as f:
    f.write(yaml_content)
def train_convnext_yolo(model_path='yolov8n.pt', data_yaml='/kaggle/working/home-fire.yaml', epochs=100):
    """Train ConvNeXt-YOLO model"""
    
    # Initialize your custom model
    weights_path = download_convnext_weights("convnext_tiny_1k")
    # Create model with frozen backbone
    model = ConvNextYOLO(yolo_path="/kaggle/working/yolov8n.pt", num_classes=2, convnext_weights_path=weights_path, freeze_backbone=True)

    
    # Training configuration
    train_config = {
        'data': data_yaml,
        'epochs': epochs,
        'imgsz': 640,
        'batch': 16,  # Adjust based on your GPU memory
        'lr0': 0.01,  # Initial learning rate
        'lrf': 0.01,  # Final learning rate
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 3,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        'box': 7.5,  # Box loss gain
        'cls': 0.5,  # Classification loss gain
        'dfl': 1.5,  # DFL loss gain
        'save': True,
        'save_period': 10,  # Save checkpoint every 10 epochs
        'cache': False,  # Set to True if you have enough RAM
        'device': '0',  # GPU device, set to 'cpu' if no GPU
        'workers': 8,
        'project': 'runs/detect',
        'name': 'convnext_yolo',
        'exist_ok': True,
        'pretrained': False,  # We're using custom model
        'optimizer': 'AdamW',
        'verbose': True,
        'seed': 0,
        'deterministic': True,
        'single_cls': False,
        'rect': False,
        'cos_lr': True,
        'close_mosaic': 10,  # Disable mosaic augmentation for last 10 epochs
        'resume': False,
        'amp': True,  # Automatic Mixed Precision
        'fraction': 1.0,
        'profile': False,
        'freeze': None,  # We handle freezing in our model
    }
    
    # Create trainer
    from ultralytics.models.yolo.detect import DetectionTrainer
    
    trainer = DetectionTrainer(overrides=train_config)
    trainer.model = model  # Use our custom model
    
    # Start training
    trainer.train()
    
    print("🎉 Training completed!")
    return trainer

# 3. ALTERNATIVE: SIMPLER TRAINING WITH ULTRALYTICS
def train_simple_method():

    
    print("Simple Integration Approach")
    print("=" * 50)
    
    # Setup
    weights_path = download_convnext_weights("convnext_tiny_1k")
    
    # Create a custom model configuration
    def create_custom_model():
        # Load base YOLO
        yolo = YOLO("yolov8n.pt")
        
        # Replace backbone
        yolo.model.model[0] = ConvNextBackbone(
            weights_path=weights_path,
            pretrained=True
        )
        
        # Freeze backbone
        for param in yolo.model.model[0].parameters():
            param.requires_grad = False
        
        return yolo
    
    # Create and train
    model = create_custom_model()
    
    # Train with standard YOLO interface
    results = model.train(
        data='/kaggle/working/home-fire.yaml',
        epochs=50,
        imgsz=640,
        batch=8,
        lr0=0.001,
        optimizer='AdamW',
        project='convnext_yolo',
        name='fire_detection',
        device=0 if torch.cuda.is_available() else 'cpu',
        verbose=True
    )
    
    return results, model 
    
    print("Simple Integration Approach")
    print("=" * 50)
    
    # Setup
    setup_yaml_config()
    weights_path = download_convnext_weights("convnext_tiny_1k")
    
    # Create a custom model configuration
    def create_custom_model():
        # Load base YOLO
        yolo = YOLO("yolov8n.pt")
        
        # Replace backbone
        yolo.model.model[0] = ConvNextBackbone(
            weights_path=weights_path,
            pretrained=True
        )
        
        # Freeze backbone
        for param in yolo.model.model[0].parameters():
            param.requires_grad = False
        
        return yolo
    
    # Create and train
    model = create_custom_model()
    
    # Train with standard YOLO interface
    results = model.train(
        data='/kaggle/working/home-fire.yaml',
        epochs=50,
        imgsz=640,
        batch=8,
        lr0=0.001,
        optimizer='AdamW',
        project='convnext_yolo',
        name='fire_detection',
        device=0 if torch.cuda.is_available() else 'cpu',
        verbose=True
    )
    
    return results, model

# 4. PROGRESSIVE TRAINING STRATEGY
def progressive_training():
    """Alternative: Manual training loop with custom components"""
    
    print("Manual training approach...")
    
    # Create model
    weights_path = download_convnext_weights("convnext_tiny_1k")
    model = WorkingConvNextYOLO(
        yolo_path="yolov8n.pt",
        num_classes=2,
        convnext_weights_path=weights_path,
        freeze_backbone=True
    )
    
    # Load dataset using YOLO's data loading
    from ultralytics.data import build_dataloader
    from ultralytics.cfg import get_cfg
    
    # Load config
    cfg = get_cfg(overrides={'data': '/kaggle/working/home-fire.yaml'})
    
    # Create data loaders
    train_loader = build_dataloader(
        dataset_path='/kaggle/input/home-fire-dataset/train',
        imgsz=640,
        batch_size=8,
        stride=32,
        single_cls=False,
        hyp=cfg,
        augment=True,
        cache=False,
        workers=2,
        shuffle=True,
        rank=-1
    )
    
    val_loader = build_dataloader(
        dataset_path='/kaggle/input/home-fire-dataset/val',
        imgsz=640,
        batch_size=8,
        stride=32,
        single_cls=False,
        hyp=cfg,
        augment=False,
        cache=False,
        workers=2,
        shuffle=False,
        rank=-1
    )
    
    # Setup training
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Optimizer with different LRs
    backbone_params = []
    other_params = []
    
    for name, param in model.named_parameters():
        if 'backbone' in name:
            backbone_params.append(param)
        else:
            other_params.append(param)
    
    optimizer = torch.optim.AdamW([
        {'params': backbone_params, 'lr': 1e-5},  # Lower for pretrained
        {'params': other_params, 'lr': 1e-3}     # Higher for new layers
    ], weight_decay=1e-4)
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)
    
    # Training loop
    num_epochs = 50
    best_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        for batch_idx, batch in enumerate(train_loader):
            images = batch['img'].to(device).float() / 255.0
            targets = batch['batch_idx'], batch['cls'], batch['bboxes']
            
            optimizer.zero_grad()
            
            # Forward pass
            predictions = model(images)
            
            # Calculate loss (you'll need to implement proper YOLO loss)
            # For now, using a simple placeholder
            if isinstance(predictions, (list, tuple)):
                loss = sum(pred.mean() for pred in predictions if pred.numel() > 0)
            else:
                loss = predictions.mean()
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            
            if batch_idx % 20 == 0:
                print(f"  Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")
        
        scheduler.step()
        avg_loss = epoch_loss / len(train_loader)
        
        # Save best model
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
            }, 'best_convnext_yolo_manual.pth')
            print(f"Best model saved! Loss: {avg_loss:.4f}")
        
        print(f"Epoch {epoch+1} completed, Avg Loss: {avg_loss:.4f}")

# 5. VALIDATION AND TESTING
def validate_model(model_path, data_yaml='/kaggle/working/home-fire.yaml'):
    """Validate trained model"""
    
    model = YOLO(model_path)
    results = model.val(
        data=data_yaml,
        imgsz=640,
        batch=1,
        conf=0.25,
        iou=0.6,
        device=0
    )
    
    print(f"mAP50: {results.box.map50}")
    print(f"mAP50-95: {results.box.map}")
    
    return results

# 6. MAIN EXECUTION
if __name__ == "__main__":
    # Step 1: Setup dataset
    data_yaml = '/kaggle/working/home-fire.yaml'
    
    # Step 2: Choose training method
    print("Choose training method:")
    print("1. Custom ConvNeXt-YOLO training")
    print("2. Progressive training (recommended)")
    print("3. Simple method")
    
    choice = input("Enter choice (1-3): ")
    
    if choice == "1":
        trainer = train_convnext_yolo(data_yaml=data_yaml)
    elif choice == "2":
        progressive_training()
    else:
        results = train_simple_method()
    
    print("🎯 Training pipeline complete!")

                                                                            

Choose training method:
1. Custom ConvNeXt-YOLO training
2. Progressive training (recommended)
3. Simple method


Enter choice (1-3):  5


Simple Integration Approach
✅ Found existing weights: convnext_tiny_1k.pth
🔄 Loading pretrained weights from convnext_tiny_1k.pth
✅ Pretrained weights loaded successfully!
✅ ConvNext Tiny backbone ready for YOLO integration
📊 Feature channels: [192, 384, 768] → [256, 512, 1024]
Ultralytics 8.3.197 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/home-fire.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_w

KeyError: 'model.0.conv.weight'

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import glob
import os
import cv2
import random
from pathlib import Path

import os, glob, random
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
from PIL import Image

class FireDetectionDataset(Dataset):
    """Custom dataset for fire detection (YOLO-style labels)."""
    
    def __init__(self, images_dir, labels_dir, img_size=640, augment=False):
        self.img_size = img_size
        self.augment = augment
        
        # Collect image paths
        image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']
        self.image_paths = []
        for ext in image_extensions:
            self.image_paths.extend(glob.glob(os.path.join(images_dir, ext)))
        
        self.labels_dir = labels_dir
        print(f"Found {len(self.image_paths)} images in {images_dir}")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        
        # --- Load image ---
        image = cv2.imread(img_path)
        if image is None:
            image = np.array(Image.open(img_path).convert('RGB'))
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        h0, w0 = image.shape[:2]
        image = cv2.resize(image, (self.img_size, self.img_size))
        h, w = image.shape[:2]

        # --- Load labels ---
        targets = {"boxes": torch.empty((0, 4)), "labels": torch.empty((0,), dtype=torch.long)}
        if self.labels_dir is not None:
            label_file = os.path.splitext(os.path.basename(img_path))[0] + ".txt"
            label_path = os.path.join(self.labels_dir, label_file)
            if os.path.exists(label_path):
                boxes, labels = [], []
                with open(label_path, "r") as f:
                    for line in f.readlines():
                        cls, x, y, bw, bh = map(float, line.strip().split())
                        labels.append(int(cls))
                        boxes.append([
                            x * w, y * h,      # center x,y (scaled to resized image)
                            bw * w, bh * h     # width, height (scaled)
                        ])
                if boxes:
                    targets["boxes"] = torch.tensor(boxes, dtype=torch.float32)
                    targets["labels"] = torch.tensor(labels, dtype=torch.long)

        # --- Augmentation ---
        if self.augment:
            if random.random() > 0.5:  # horizontal flip
                image = cv2.flip(image, 1)
                if targets["boxes"].numel() > 0:
                    targets["boxes"][:, 0] = w - targets["boxes"][:, 0]  # flip x center

        # --- Final conversion ---
        image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
        
        return image, targets


def train_working_convnext_yolo(dataset_path, epochs=30, batch_size=4, learning_rate=0.001):
    """Train the WorkingConvNextYOLO model"""
    
    print("Training WorkingConvNextYOLO Model")
    print("=" * 50)
    
    # Device setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Download ConvNeXt weights
    print("Downloading ConvNeXt weights...")
    weights_path = download_convnext_weights("convnext_tiny_1k")
    
    # Create datasets
    print("Creating datasets...")
    train_dataset, val_dataset = create_fire_datasets(dataset_path)
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=2,
        pin_memory=True,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2,
        pin_memory=True,
        drop_last=False
    )
    
    print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")
    
    # Create model
    print("Creating WorkingConvNextYOLO model...")
    model = WorkingConvNextYOLO(
        yolo_path="yolov8n.pt",
        num_classes=2,  # fire and smoke
        convnext_weights_path=weights_path,
        freeze_backbone=True
    )
    
    model = model.to(device)
    model.train()
    
    # Setup optimizer with different learning rates
    backbone_params = []
    other_params = []
    
    for name, param in model.named_parameters():
        if param.requires_grad:
            if 'backbone' in name.lower() or 'convnext' in name.lower():
                backbone_params.append(param)
            else:
                other_params.append(param)
    
    # Optimizer with different learning rates
    optimizer = torch.optim.AdamW([
        {'params': backbone_params, 'lr': learning_rate * 0.1, 'weight_decay': 1e-4},  # Lower LR for backbone
        {'params': other_params, 'lr': learning_rate, 'weight_decay': 1e-4}  # Higher LR for head
    ])
    
    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)
    
    # Training variables
    best_loss = float('inf')
    train_losses = []
    val_losses = []
    
    print(f"Starting training for {epochs} epochs...")
    print(f"Batch size: {batch_size}, Learning rate: {learning_rate}")
    
    # Training loop
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        print("-" * 40)
        
        # Training phase
        model.train()
        epoch_train_loss = 0.0
        num_train_batches = 0
        
        for batch_idx, images in enumerate(train_loader):
            images = images.to(device, non_blocking=True)
            
            # Zero gradients
            optimizer.zero_grad()
            
            try:
                # Forward pass
                outputs = model(images)
                
                # Compute loss (simplified loss function)
                # In practice, you'd want to implement proper YOLO loss with ground truth labels
                if isinstance(outputs, (list, tuple)):
                    # Handle multiple outputs (typical for YOLO)
                    loss = torch.tensor(0.0, requires_grad=True).to(device)
                    for output in outputs:
                        if torch.is_tensor(output) and output.requires_grad:
                            # Simple loss based on output magnitude (placeholder)
                            loss = loss + torch.mean(torch.abs(output)) * 0.1
                else:
                    # Single output
                    loss = torch.mean(torch.abs(outputs)) * 0.1
                
                # Add regularization loss
                l2_loss = torch.tensor(0.0).to(device)
                for param in model.parameters():
                    if param.requires_grad:
                        l2_loss = l2_loss + torch.norm(param, 2) ** 2
                
                total_loss = loss + 1e-6 * l2_loss
                
                # Backward pass
                total_loss.backward()
                
                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
                
                # Optimizer step
                optimizer.step()
                
                epoch_train_loss += total_loss.item()
                num_train_batches += 1
                
                # Print progress
                if batch_idx % 20 == 0:
                    print(f"  Batch {batch_idx:3d}/{len(train_loader)} | Loss: {total_loss.item():.6f}")
                
            except Exception as e:
                print(f"  Error in batch {batch_idx}: {str(e)}")
                continue
        
        # Calculate average training loss
        if num_train_batches > 0:
            avg_train_loss = epoch_train_loss / num_train_batches
            train_losses.append(avg_train_loss)
        else:
            avg_train_loss = float('inf')
            train_losses.append(avg_train_loss)
        
        # Validation phase
        model.eval()
        epoch_val_loss = 0.0
        num_val_batches = 0
        
        with torch.no_grad():
            for batch_idx, images in enumerate(val_loader):
                images = images.to(device, non_blocking=True)
                
                try:
                    outputs = model(images)
                    
                    # Validation loss (same as training loss)
                    if isinstance(outputs, (list, tuple)):
                        val_loss = torch.tensor(0.0).to(device)
                        for output in outputs:
                            if torch.is_tensor(output):
                                val_loss = val_loss + torch.mean(torch.abs(output)) * 0.1
                    else:
                        val_loss = torch.mean(torch.abs(outputs)) * 0.1
                    
                    epoch_val_loss += val_loss.item()
                    num_val_batches += 1
                    
                except Exception as e:
                    print(f"  Validation error in batch {batch_idx}: {str(e)}")
                    continue
        
        # Calculate average validation loss
        if num_val_batches > 0:
            avg_val_loss = epoch_val_loss / num_val_batches
            val_losses.append(avg_val_loss)
        else:
            avg_val_loss = float('inf')
            val_losses.append(avg_val_loss)
        
        # Update learning rate
        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        
        # Save best model
        if avg_train_loss < best_loss:
            best_loss = avg_train_loss
            checkpoint = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'train_loss': avg_train_loss,
                'val_loss': avg_val_loss,
                'convnext_weights_path': weights_path
            }
            torch.save(checkpoint, 'best_working_convnext_yolo.pth')
            print(f"  *** Best model saved! Train Loss: {avg_train_loss:.6f}")
        
        # Print epoch summary
        print(f"  Epoch Summary:")
        print(f"    Train Loss: {avg_train_loss:.6f}")
        print(f"    Val Loss:   {avg_val_loss:.6f}")
        print(f"    Learning Rate: {current_lr:.8f}")
        print(f"    Best Loss: {best_loss:.6f}")
    
    print("\nTraining completed!")
    print(f"Best training loss: {best_loss:.6f}")
    print("Model saved as: best_working_convnext_yolo.pth")
    
    return model, train_losses, val_losses

def load_trained_model(checkpoint_path='best_working_convnext_yolo.pth'):
    """Load a trained WorkingConvNextYOLO model"""
    
    print(f"Loading trained model from {checkpoint_path}")
    
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    
    # Create model
    model = WorkingConvNextYOLO(
        yolo_path="yolov8n.pt",
        num_classes=2,
        convnext_weights_path=checkpoint.get('convnext_weights_path'),
        freeze_backbone=True
    )
    
    # Load state dict
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"Model loaded successfully!")
    print(f"Trained for {checkpoint['epoch']} epochs")
    print(f"Final train loss: {checkpoint['train_loss']:.6f}")
    
    return model

def test_model_inference(model, test_image_path):
    """Test the trained model on a single image"""
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    
    # Load and preprocess image
    image = cv2.imread(test_image_path)
    if image is None:
        image = np.array(Image.open(test_image_path).convert('RGB'))
    else:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Resize and normalize
    image = cv2.resize(image, (640, 640))
    image_tensor = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
    image_tensor = image_tensor.unsqueeze(0).to(device)
    
    # Inference
    with torch.no_grad():
        outputs = model(image_tensor)
    
    print(f"Model output type: {type(outputs)}")
    if isinstance(outputs, (list, tuple)):
        print(f"Number of outputs: {len(outputs)}")
        for i, out in enumerate(outputs):
            if torch.is_tensor(out):
                print(f"  Output {i}: {out.shape}")
    else:
        print(f"Output shape: {outputs.shape}")
    
    return outputs


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math


def bbox_ciou(pred_boxes, target_boxes, eps=1e-7):
    """CIoU between pred and target boxes (x,y,w,h)."""
    # Convert to corners
    pred_x1 = pred_boxes[:, 0] - pred_boxes[:, 2] / 2
    pred_y1 = pred_boxes[:, 1] - pred_boxes[:, 3] / 2
    pred_x2 = pred_boxes[:, 0] + pred_boxes[:, 2] / 2
    pred_y2 = pred_boxes[:, 1] + pred_boxes[:, 3] / 2

    targ_x1 = target_boxes[:, 0] - target_boxes[:, 2] / 2
    targ_y1 = target_boxes[:, 1] - target_boxes[:, 3] / 2
    targ_x2 = target_boxes[:, 0] + target_boxes[:, 2] / 2
    targ_y2 = target_boxes[:, 1] + target_boxes[:, 3] / 2

    # Intersection
    inter_x1 = torch.max(pred_x1, targ_x1)
    inter_y1 = torch.max(pred_y1, targ_y1)
    inter_x2 = torch.min(pred_x2, targ_x2)
    inter_y2 = torch.min(pred_y2, targ_y2)
    inter_area = (inter_x2 - inter_x1).clamp(0) * (inter_y2 - inter_y1).clamp(0)

    # Union
    pred_area = (pred_x2 - pred_x1).clamp(0) * (pred_y2 - pred_y1).clamp(0)
    targ_area = (targ_x2 - targ_x1).clamp(0) * (targ_y2 - targ_y1).clamp(0)
    union = pred_area + targ_area - inter_area + eps
    iou = inter_area / union

    # Enclosing box
    cw = torch.max(pred_x2, targ_x2) - torch.min(pred_x1, targ_x1)
    ch = torch.max(pred_y2, targ_y2) - torch.min(pred_y1, targ_y1)
    c2 = cw ** 2 + ch ** 2 + eps

    # Center distance
    rho2 = ((pred_boxes[:, 0] - target_boxes[:, 0]) ** 2 +
            (pred_boxes[:, 1] - target_boxes[:, 1]) ** 2)

    # Aspect ratio penalty
    v = (4 / (math.pi ** 2)) * torch.pow(
        torch.atan(target_boxes[:, 2] / (target_boxes[:, 3] + eps)) -
        torch.atan(pred_boxes[:, 2] / (pred_boxes[:, 3] + eps)), 2
    )
    with torch.no_grad():
        alpha = v / (v - iou + (1 + eps))

    ciou = iou - (rho2 / c2 + alpha * v)
    return ciou



class YOLOLoss(nn.Module):
    def __init__(self, num_classes=1, box_gain=7.5, obj_gain=1.0, cls_gain=0.5, iou_threshold=0.1):
        super().__init__()
        self.num_classes = num_classes
        self.box_gain = box_gain
        self.obj_gain = obj_gain
        self.cls_gain = cls_gain
        self.iou_threshold = iou_threshold

        self.bce_obj = nn.BCEWithLogitsLoss(reduction="mean")
        self.bce_cls = nn.BCEWithLogitsLoss(reduction="mean")

    def forward(self, preds, targets):
        """
        preds: [B, 6, N] where 6 = (x,y,w,h,obj,cls)
        targets: list of dicts, each with "boxes" [M,4] and "labels" [M]
        """
        device = preds.device
        B, _, N = preds.shape

        # Split predictions
        pred_boxes = preds[:, :4, :].permute(0, 2, 1)  # [B, N, 4]
        pred_obj   = preds[:, 4, :]                    # [B, N]
        pred_cls   = preds[:, 5:, :].permute(0, 2, 1)  # [B, N, num_classes]

        total_lbox, total_lobj, total_lcls = 0.0, 0.0, 0.0

        for i in range(B):
            boxes_i  = pred_boxes[i]   # [N,4]
            obj_i    = pred_obj[i]     # [N]
            cls_i    = pred_cls[i]     # [N, num_classes]

            targets_i = targets[i]
            target_boxes  = targets_i["boxes"].to(device)   # [M,4]
            target_labels = targets_i["labels"].to(device)  # [M]

            M = target_boxes.shape[0]

            if M == 0:
                # No targets → all objectness = 0
                total_lobj += self.bce_obj(obj_i, torch.zeros_like(obj_i)) * self.obj_gain
                continue

            # Match predictions to GT via IoU
            ious = bbox_iou_matrix(boxes_i, target_boxes)  # [N, M]
            best_iou, best_idx = ious.max(dim=1)           # [N]
            pos_mask = best_iou > self.iou_threshold       # [N]

            matched_boxes  = target_boxes[best_idx[pos_mask]]
            matched_labels = target_labels[best_idx[pos_mask]]

            # --- Box loss (CIoU) ---
            if pos_mask.sum() > 0:
                l_box = (1 - bbox_ciou(boxes_i[pos_mask], matched_boxes)).mean() * self.box_gain
                l_cls = self.bce_cls(
                    cls_i[pos_mask].squeeze(-1),   # [N,1] -> [N]
                    matched_labels.float()         # [N]
                ) * self.cls_gain

            else:
                l_box = torch.tensor(0., device=device)
                l_cls = torch.tensor(0., device=device)

            # --- Objectness loss ---
            l_obj = self.bce_obj(obj_i, pos_mask.float()) * self.obj_gain

            total_lbox += l_box
            total_lobj += l_obj
            total_lcls += l_cls

        total = total_lbox + total_lobj + total_lcls

        return total, {
            "lbox": total_lbox.detach(),
            "lobj": total_lobj.detach(),
            "lcls": total_lcls.detach()
        }

def bbox_iou_matrix(box1, box2, eps=1e-7):
    """Compute IoU between two sets of boxes: box1=[N,4], box2=[M,4]"""
   
    device = box1.device
    box2 = box2.to(device)

    # Convert to corners
    box1_x1 = box1[:,0:1] - box1[:,2:3]/2
    box1_y1 = box1[:,1:2] - box1[:,3:4]/2
    box1_x2 = box1[:,0:1] + box1[:,2:3]/2
    box1_y2 = box1[:,1:2] + box1[:,3:4]/2

    box2_x1 = box2[:,0:1] - box2[:,2:3]/2
    box2_y1 = box2[:,1:2] - box2[:,3:4]/2
    box2_x2 = box2[:,0:1] + box2[:,2:3]/2
    box2_y2 = box2[:,1:2] + box2[:,3:4]/2

    # Intersection
    inter_x1 = torch.max(box1_x1, box2_x1.T)
    inter_y1 = torch.max(box1_y1, box2_y1.T)
    inter_x2 = torch.min(box1_x2, box2_x2.T)
    inter_y2 = torch.min(box1_y2, box2_y2.T)
    inter_area = (inter_x2 - inter_x1).clamp(0) * (inter_y2 - inter_y1).clamp(0)

    # Union
    area1 = (box1_x2 - box1_x1) * (box1_y2 - box1_y1)
    area2 = (box2_x2 - box2_x1) * (box2_y2 - box2_y1)
    union = area1 + area2.T - inter_area + eps

    return inter_area / union  # [N,M]

In [13]:
import os, glob, random
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
from PIL import Image

class FireDetectionDataset(Dataset):
    """Custom dataset for fire detection (YOLO-style labels)."""
    
    def __init__(self, images_dir, labels_dir, img_size=640, augment=False):
        self.img_size = img_size
        self.augment = augment
        
        # Collect image paths
        image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']
        self.image_paths = []
        for ext in image_extensions:
            self.image_paths.extend(glob.glob(os.path.join(images_dir, ext)))
        
        self.labels_dir = labels_dir
        print(f"Found {len(self.image_paths)} images in {images_dir}")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        
        # --- Load image ---
        image = cv2.imread(img_path)
        if image is None:
            image = np.array(Image.open(img_path).convert('RGB'))
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        h0, w0 = image.shape[:2]
        image = cv2.resize(image, (self.img_size, self.img_size))
        h, w = image.shape[:2]

        # --- Load labels ---
        targets = {"boxes": torch.empty((0, 4)), "labels": torch.empty((0,), dtype=torch.long)}
        if self.labels_dir is not None:
            label_file = os.path.splitext(os.path.basename(img_path))[0] + ".txt"
            label_path = os.path.join(self.labels_dir, label_file)
            if os.path.exists(label_path):
                boxes, labels = [], []
                with open(label_path, "r") as f:
                    for line in f.readlines():
                        cls, x, y, bw, bh = map(float, line.strip().split())
                        labels.append(int(cls))
                        boxes.append([
                            x * w, y * h,      # center x,y (scaled to resized image)
                            bw * w, bh * h     # width, height (scaled)
                        ])
                if boxes:
                    targets["boxes"] = torch.tensor(boxes, dtype=torch.float32)
                    targets["labels"] = torch.tensor(labels, dtype=torch.long)

        # --- Augmentation ---
        if self.augment:
            if random.random() > 0.5:  # horizontal flip
                image = cv2.flip(image, 1)
                if targets["boxes"].numel() > 0:
                    targets["boxes"][:, 0] = w - targets["boxes"][:, 0]  # flip x center

        # --- Final conversion ---
        image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
        
        return image, targets
def create_fire_datasets(dataset_base_path, img_size=640):
    train_dataset = FireDetectionDataset(
        os.path.join(dataset_base_path, 'train', 'images'),
        os.path.join(dataset_base_path, 'train', 'labels'),
        img_size=img_size, augment=True
    )
    
    val_dataset = FireDetectionDataset(
        os.path.join(dataset_base_path, 'val', 'images'),
        os.path.join(dataset_base_path, 'val', 'labels'),
        img_size=img_size, augment=False
    )
    
    test_dataset = FireDetectionDataset(
        os.path.join(dataset_base_path, 'test', 'images'),
        os.path.join(dataset_base_path, 'test', 'labels'),
        img_size=img_size, augment=False
    )
    
    return train_dataset, val_dataset, test_dataset


In [15]:
from types import SimpleNamespace
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model
dataset_path = "/kaggle/input/home-fire-dataset"
weights_path = download_convnext_weights("convnext_tiny_1k")
batch_size=8    
    # Create datasets
print("Creating datasets...")
train_dataset, val_dataset, test_dataset = create_fire_datasets(dataset_path)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=2, pin_memory=True, drop_last=True, collate_fn=lambda b: tuple(zip(*b)))

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=2, pin_memory=True, drop_last=False, collate_fn=lambda b: tuple(zip(*b)))

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=2, pin_memory=True, drop_last=False, collate_fn=lambda b: tuple(zip(*b)))
    
print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")
    
    # Create model
print("Creating WorkingConvNextYOLO model...")
model = WorkingConvNextYOLO(
        yolo_path="yolov8n.pt",
        num_classes=2,  # fire and smoke
        convnext_weights_path=weights_path,
        freeze_backbone=True
    )
model = model.to(device)
# Loss
criterion = YOLOLoss(num_classes=2)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop (simplified)
num_epochs = 10  # set how many epochs you want

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    epoch_lbox = 0.0
    epoch_lobj = 0.0
    epoch_lcls = 0.0
    num_batches = 0
    for images, targets in train_loader:  # images: [B,3,H,W], targets: list of dicts
        images = torch.stack(images).to(device) 
        
        preds = model(images)   # This is YOLO forward
        # ❗ Ensure preds are reshaped as your loss expects
        # Example: preds already come as list of feature maps [B,C,H,W]
        # Convert to [B,H,W,C]
        if isinstance(preds, (list, tuple)):
            preds = preds[0]   # ✅ now preds is [B,6,N]
        num_classes = 2
        reg_max = 16
        no = num_classes + 4 * reg_max
        B, C, H, W = preds.shape
        na = C // no  
        
        # reshape to [B, na, no, H, W]
        preds = preds.view(B, na, no, H, W)
        
        # move dims -> [B, H, W, na, no]
        preds = preds.permute(0, 3, 4, 1, 2)
        
        # flatten -> [B, N, no]
        preds = preds.reshape(B, -1, no)
        loss, loss_items = criterion(preds, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss_items)
        epoch_loss += loss.item()
        epoch_lbox += loss_items["lbox"].item()
        epoch_lobj += loss_items["lobj"].item()
        epoch_lcls += loss_items["lcls"].item()
        num_batches += 1

    # Average over batches
    avg_loss = epoch_loss / num_batches
    avg_lbox = epoch_lbox / num_batches
    avg_lobj = epoch_lobj / num_batches
    avg_lcls = epoch_lcls / num_batches
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Loss: {avg_loss:.4f}, "
          f"lbox: {avg_lbox:.4f}, "
          f"lobj: {avg_lobj:.4f}, "
          f"lcls: {avg_lcls:.4f}")


✅ Found existing weights: convnext_tiny_1k.pth
Creating datasets...
Found 3900 images in /kaggle/input/home-fire-dataset/train/images
Found 1300 images in /kaggle/input/home-fire-dataset/val/images
Found 1300 images in /kaggle/input/home-fire-dataset/test/images
Train batches: 487, Val batches: 163
Creating WorkingConvNextYOLO model...
Creating Working ConvNext-YOLO...
Original YOLO type: <class 'ultralytics.nn.tasks.DetectionModel'>
Model layers type: <class 'torch.nn.modules.container.Sequential'>
Number of layers: 23
Replacing backbone with ConvNeXt...
🔄 Loading pretrained weights from convnext_tiny_1k.pth
✅ Pretrained weights loaded successfully!
✅ ConvNext Tiny backbone ready for YOLO integration
📊 Feature channels: [192, 384, 768] → [256, 512, 1024]
swapped
Freezing ConvNeXt backbone...
Updating detection head: -> 2 classes
Found detection layer: <class 'ultralytics.nn.modules.head.Detect'>
  Updated cv2[0]: 64 -> 2
  Updated cv2[1]: 64 -> 2
  Updated cv2[2]: 64 -> 2
  Updated cv

KeyboardInterrupt: 

In [16]:

from __future__ import annotations

from copy import copy
from typing import Any

import torch

from ultralytics.data import ClassificationDataset, build_dataloader
from ultralytics.engine.trainer import BaseTrainer
from ultralytics.models import yolo
from ultralytics.nn.tasks import ClassificationModel
from ultralytics.utils import DEFAULT_CFG, LOGGER, RANK
from ultralytics.utils.plotting import plot_images, plot_results
from ultralytics.utils.torch_utils import is_parallel, strip_optimizer, torch_distributed_zero_first


class ClassificationTrainer(BaseTrainer):
    """
    A trainer class extending BaseTrainer for training image classification models.

    This trainer handles the training process for image classification tasks, supporting both YOLO classification models
    and torchvision models with comprehensive dataset handling and validation.

    Attributes:
        model (ClassificationModel): The classification model to be trained.
        data (dict[str, Any]): Dictionary containing dataset information including class names and number of classes.
        loss_names (list[str]): Names of the loss functions used during training.
        validator (ClassificationValidator): Validator instance for model evaluation.

    Methods:
        set_model_attributes: Set the model's class names from the loaded dataset.
        get_model: Return a modified PyTorch model configured for training.
        setup_model: Load, create or download model for classification.
        build_dataset: Create a ClassificationDataset instance.
        get_dataloader: Return PyTorch DataLoader with transforms for image preprocessing.
        preprocess_batch: Preprocess a batch of images and classes.
        progress_string: Return a formatted string showing training progress.
        get_validator: Return an instance of ClassificationValidator.
        label_loss_items: Return a loss dict with labelled training loss items.
        plot_metrics: Plot metrics from a CSV file.
        final_eval: Evaluate trained model and save validation results.
        plot_training_samples: Plot training samples with their annotations.

    Examples:
        Initialize and train a classification model
        >>> from ultralytics.models.yolo.classify import ClassificationTrainer
        >>> args = dict(model="yolo11n-cls.pt", data="imagenet10", epochs=3)
        >>> trainer = ClassificationTrainer(overrides=args)
        >>> trainer.train()
    """

    def __init__(self, cfg=DEFAULT_CFG, overrides: dict[str, Any] | None = None, _callbacks=None):
        """
        Initialize a ClassificationTrainer object.

        This constructor sets up a trainer for image classification tasks, configuring the task type and default
        image size if not specified.

        Args:
            cfg (dict[str, Any], optional): Default configuration dictionary containing training parameters.
            overrides (dict[str, Any], optional): Dictionary of parameter overrides for the default configuration.
            _callbacks (list[Any], optional): List of callback functions to be executed during training.

        Examples:
            Create a trainer with custom configuration
            >>> from ultralytics.models.yolo.classify import ClassificationTrainer
            >>> args = dict(model="yolo11n-cls.pt", data="imagenet10", epochs=3)
            >>> trainer = ClassificationTrainer(overrides=args)
            >>> trainer.train()
        """
        if overrides is None:
            overrides = {}
        overrides["task"] = "classify"
        if overrides.get("imgsz") is None:
            overrides["imgsz"] = 224
        super().__init__(cfg, overrides, _callbacks)

    def set_model_attributes(self):
        """Set the YOLO model's class names from the loaded dataset."""
        self.model.names = self.data["names"]

    def get_model(self, cfg=None, weights=None, verbose: bool = True):
        """
        Return a modified PyTorch model configured for training YOLO classification.

        Args:
            cfg (Any, optional): Model configuration.
            weights (Any, optional): Pre-trained model weights.
            verbose (bool, optional): Whether to display model information.

        Returns:
            (ClassificationModel): Configured PyTorch model for classification.
        """
        model = ClassificationModel(cfg, nc=self.data["nc"], ch=self.data["channels"], verbose=verbose and RANK == -1)
        if weights:
            model.load(weights)

        for m in model.modules():
            if not self.args.pretrained and hasattr(m, "reset_parameters"):
                m.reset_parameters()
            if isinstance(m, torch.nn.Dropout) and self.args.dropout:
                m.p = self.args.dropout  # set dropout
        for p in model.parameters():
            p.requires_grad = True  # for training
        return model

    def setup_model(self):
        """
        Load, create or download model for classification tasks.

        Returns:
            (Any): Model checkpoint if applicable, otherwise None.
        """
        import torchvision  # scope for faster 'import ultralytics'

        if str(self.model) in torchvision.models.__dict__:
            self.model = torchvision.models.__dict__[self.model](
                weights="IMAGENET1K_V1" if self.args.pretrained else None
            )
            ckpt = None
        else:
            ckpt = super().setup_model()
        ClassificationModel.reshape_outputs(self.model, self.data["nc"])
        return ckpt

    def build_dataset(self, img_path: str, mode: str = "train", batch=None):
        """
        Create a ClassificationDataset instance given an image path and mode.

        Args:
            img_path (str): Path to the dataset images.
            mode (str, optional): Dataset mode ('train', 'val', or 'test').
            batch (Any, optional): Batch information (unused in this implementation).

        Returns:
            (ClassificationDataset): Dataset for the specified mode.
        """
        return ClassificationDataset(root=img_path, args=self.args, augment=mode == "train", prefix=mode)

    def get_dataloader(self, dataset_path: str, batch_size: int = 16, rank: int = 0, mode: str = "train"):
        """
        Return PyTorch DataLoader with transforms to preprocess images.

        Args:
            dataset_path (str): Path to the dataset.
            batch_size (int, optional): Number of images per batch.
            rank (int, optional): Process rank for distributed training.
            mode (str, optional): 'train', 'val', or 'test' mode.

        Returns:
            (torch.utils.data.DataLoader): DataLoader for the specified dataset and mode.
        """
        with torch_distributed_zero_first(rank):  # init dataset *.cache only once if DDP
            dataset = self.build_dataset(dataset_path, mode)

        loader = build_dataloader(dataset, batch_size, self.args.workers, rank=rank)
        # Attach inference transforms
        if mode != "train":
            if is_parallel(self.model):
                self.model.module.transforms = loader.dataset.torch_transforms
            else:
                self.model.transforms = loader.dataset.torch_transforms
        return loader

    def preprocess_batch(self, batch: dict[str, torch.Tensor]) -> dict[str, torch.Tensor]:
        """Preprocess a batch of images and classes."""
        batch["img"] = batch["img"].to(self.device, non_blocking=True)
        batch["cls"] = batch["cls"].to(self.device, non_blocking=True)
        return batch

    def progress_string(self) -> str:
        """Return a formatted string showing training progress."""
        return ("\n" + "%11s" * (4 + len(self.loss_names))) % (
            "Epoch",
            "GPU_mem",
            *self.loss_names,
            "Instances",
            "Size",
        )

    def get_validator(self):
        """Return an instance of ClassificationValidator for validation."""
        self.loss_names = ["loss"]
        return yolo.classify.ClassificationValidator(
            self.test_loader, self.save_dir, args=copy(self.args), _callbacks=self.callbacks
        )

    def label_loss_items(self, loss_items: torch.Tensor | None = None, prefix: str = "train"):
        """
        Return a loss dict with labelled training loss items tensor.

        Args:
            loss_items (torch.Tensor, optional): Loss tensor items.
            prefix (str, optional): Prefix to prepend to loss names.

        Returns:
            keys (list[str]): List of loss keys if loss_items is None.
            loss_dict (dict[str, float]): Dictionary of loss items if loss_items is provided.
        """
        keys = [f"{prefix}/{x}" for x in self.loss_names]
        if loss_items is None:
            return keys
        loss_items = [round(float(loss_items), 5)]
        return dict(zip(keys, loss_items))

    def plot_metrics(self):
        """Plot metrics from a CSV file."""
        plot_results(file=self.csv, classify=True, on_plot=self.on_plot)  # save results.png

    def final_eval(self):
        """Evaluate trained model and save validation results."""
        for f in self.last, self.best:
            if f.exists():
                strip_optimizer(f)  # strip optimizers
                if f is self.best:
                    LOGGER.info(f"\nValidating {f}...")
                    self.validator.args.data = self.args.data
                    self.validator.args.plots = self.args.plots
                    self.metrics = self.validator(model=f)
                    self.metrics.pop("fitness", None)
                    self.run_callbacks("on_fit_epoch_end")

    def plot_training_samples(self, batch: dict[str, torch.Tensor], ni: int):
        """
        Plot training samples with their annotations.

        Args:
            batch (dict[str, torch.Tensor]): Batch containing images and class labels.
            ni (int): Number of iterations.
        """
        batch["batch_idx"] = torch.arange(len(batch["img"]))  # add batch index for plotting
        plot_images(
            labels=batch,
            fname=self.save_dir / f"train_batch{ni}.jpg",
            on_plot=self.on_plot,
        )

In [ ]:
from ultralytics.models.yolo.detect.train import DetectionTrainer
from types import SimpleNamespace
from ultralytics.utils.loss import v8DetectionLoss

class ConvNextYOLOTrainer(DetectionTrainer):
    def get_model(self, cfg=None, weights=None, verbose=True):
        """
        Override YOLO model loading with our custom ConvNeXt backbone.
        """
        # Initialize your custom YOLO model
        weights_path = download_convnext_weights("convnext_tiny_1k")
        model = WorkingConvNextYOLO(
            yolo_path="yolov8n.pt",
            num_classes=self.data["nc"],     # number of dataset classes
            convnext_weights_path = weights_path,      # or your convnext pretrained weights
            freeze_backbone=True             # freeze backbone if needed
        ).yolo

        
        keys = ["box", "cls", "dfl", "pose", "kobj"]
        hyp_dict = {k: getattr(self.args, k) for k in keys if hasattr(self.args, k)}
        print("⚠️ Building hyp from args:", hyp_dict)
        self.hyp = SimpleNamespace(**hyp_dict)
            
        print("done#########################")
        # Load weights if provided
        
        print("done2#########################################")
         # Build SimpleNamespace for hyp
        keys = ["box", "cls", "dfl", "pose", "kobj"]
        hyp_dict = {k: getattr(self.args, k) for k in keys if hasattr(self.args, k)}
        hyp_ns = SimpleNamespace(**hyp_dict)

        # Attach hyp to the model (NOT to args)
        model.model.args = hyp_ns
        print(model.args)
        # Initialize loss so the trainer can call it
        model.init_criterion()  # ensures self.criterion exists
        
        return model
import torch, gc
gc.collect()
torch.cuda.empty_cache()
args = dict(
    model="yolov8n.pt",   # dummy, will be replaced by get_model()
    data="/kaggle/working/home-fire.yaml",  # your dataset yaml
    epochs=50,
    imgsz=640,
    batch=8
)

trainer = ConvNextYOLOTrainer(overrides=args)
print("trainer is ready")

# Build hyp from args manually
# Do NOT assign to trainer.args.hyp
# trainer.args.hyp = trainer.hyp  <-- remove this line


trainer.train()

Ultralytics 8.3.197 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/home-fire.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=T